# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # SCF convergence criterion
                   'scf_converger' : [1], # converger used for scf loop
                   }

output = {
'molid': [0,1], # index of the molecules for which MD output is written
'prefix': f'Outputs/3_MD_BOMD_Basic',
'print every': 1, # print output every N steps 
"xyz": 1,    # write .xyz file every N steps; 0 disables (default)
# write an HDF5 file for each molecule
"h5": {
    "data": 1,      # write temp, energies, excitations, MO, etc. every N steps; 0 disables
    "coordinates": 1,   # write coords; 0 disables
    "velocities": 1,   # write velocities; 0 disables
    "forces": 1,   # write forces; 0 disables
    "write_mo": False, # write HOMO-LUMO gap
    },
"checkpoint every": 100, # write a checkpoint file to restart calculations from every N steps
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
_ = md.run(molecule, steps=10)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   396.16   2.048326e-01 -4.755799e+02 -4.753750e+02 ||    540.80   2.796143e-01 -1.092097e+02 -1.089301e+02 || 
     2   378.43   1.956660e-01 -4.755706e+02 -4.753749e+02 ||    678.08   3.505955e-01 -1.092814e+02 -1.089309e+02 || 
     3   349.19   1.805432e-01 -4.755554e+02 -4.753748e+02 ||    789.80   4.083569e-01 -1.093400e+02 -1.089316e+02 || 
     4   312.03   1.613320e-01 -4.755360e+02 -4.753747e+02 ||    854.91   4.420220e-01 -1.093742e+02 -1.089322e+02 || 
     5   271.19   1.402165e-01 -4.755148e+02 -4.753745e+02 ||    857.70   4.434668e-01 -1.093759e+02 -1.089325e+02 || 
     6   230.82   1.193433e-01 -4.754938e+02 -4.753744e+02 ||    792.53   4.097702e-01 -1.093421e+02 -1.089323e+02 || 
     7   194.37   1.004972e-01 -4.754749e+02 -4.753744e+02 ||    667.52   3.451344e-01 -1.092768e+02 -1.089316e+02 || 
     8   164.13   8.486361e-02 -4.754592e+02 -4.753743e+02 ||    505.32   2.612705e-01 -1.091918e+02 -1.089305e+

### *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created
### *MD_BOMD_BASIC.0.h5* and *MD_BOMD_BASIC.1.h5* are HDF5 files that contain all the data of the molecular dynamics trajectory

## Excited-state dynamics on a particular excited state.
### Excited-state dynamics are performed in the same way as ground-state BOMD. The only difference is that, in seqm_parameters, you need to provide the excited-state input parameters, and choose which excited state (active_state > 0) to run on.

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[8,6,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [1.80,  1.03,  0.00],
                   [1.80, -1.03,  0.00]
                  ]
                 ], device=device)

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  
                   'scf_eps' : 1.0e-7,  
                   'scf_converger' : [1], # converger used for scf loop
                   'excited_states': {'n_states':3},
                   'active_state': 1,
                   }

output = {
'molid': [0,1], # index of the molecules for which MD output is written
'prefix': f'Outputs/3_MD_BOMD_Excited',
'print every': 1, # print output every N steps 
"xyz": 1,    # write .xyz file every N steps; 0 disables (default)
# write an HDF5 file for each molecule
"h5": {
    "data": 1,      # write temp, energies, excitations, MO, etc. every N steps; 0 disables
    "coordinates": 1,   # write coords; 0 disables
    "velocities": 1,   # write velocities; 0 disables
    "forces": 1,   # write forces; 0 disables
    "write_mo": False, # write HOMO-LUMO gap
    },
"checkpoint every": 100, # write a checkpoint file to restart calculations from every N steps
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, steps=10)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   405.82   2.098231e-01 -4.727443e+02 -4.725345e+02 ||    421.66   2.180145e-01 -4.724303e+02 -4.722123e+02 || 
     2   405.73   2.097805e-01 -4.727442e+02 -4.725344e+02 ||    476.09   2.461593e-01 -4.724588e+02 -4.722127e+02 || 
     3   401.96   2.078271e-01 -4.727422e+02 -4.725343e+02 ||    552.19   2.855039e-01 -4.724987e+02 -4.722132e+02 || 
     4   396.75   2.051346e-01 -4.727394e+02 -4.725343e+02 ||    635.21   3.284286e-01 -4.725421e+02 -4.722137e+02 || 
     5   391.93   2.026457e-01 -4.727369e+02 -4.725342e+02 ||    709.71   3.669504e-01 -4.725811e+02 -4.722142e+02 || 
     6   388.61   2.009279e-01 -4.727352e+02 -4.725342e+02 ||    762.92   3.944580e-01 -4.726089e+02 -4.722144e+02 || 
     7   387.04   2.001165e-01 -4.727344e+02 -4.725343e+02 ||    787.55   4.071927e-01 -4.726217e+02 -4.722145e+02 || 
     8   386.65   1.999158e-01 -4.727342e+02 -4.725343e+02 ||    783.09   4.048914e-01 -4.726192e+02 -4.722143e+

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output = {
'molid': [0,1],
'prefix': 'Outputs/3_MD_BOMD_Basic_E_cntrl',
'print every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "write_mo": False,
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, steps=10, control_energy_shift=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   302.06   1.561751e-01 -4.742618e+02 -4.741056e+02 ||    383.97   1.985263e-01 -1.084141e+02 -1.082156e+02 || 
     2   242.59   1.254306e-01 -4.742310e+02 -4.741056e+02 ||    443.57   2.293453e-01 -1.084450e+02 -1.082156e+02 || 
     3   226.70   1.172152e-01 -4.742228e+02 -4.741056e+02 ||    570.70   2.950730e-01 -1.085107e+02 -1.082156e+02 || 
     4   263.28   1.361270e-01 -4.742417e+02 -4.741056e+02 ||    752.95   3.893032e-01 -1.086049e+02 -1.082156e+02 || 
     5   363.19   1.877848e-01 -4.742934e+02 -4.741056e+02 ||    974.09   5.036434e-01 -1.087193e+02 -1.082156e+02 || 
     6   535.39   2.768156e-01 -4.743824e+02 -4.741056e+02 ||   1214.65   6.280239e-01 -1.088436e+02 -1.082156e+02 || 
     7   782.05   4.043499e-01 -4.745099e+02 -4.741056e+02 ||   1452.43   7.509643e-01 -1.089666e+02 -1.082156e+02 || 
     8  1094.70   5.660039e-01 -4.746716e+02 -4.741056e+02 ||   1662.82   8.597428e-01 -1.090754e+02 -1.082156e+

## Scale velocities to control temperature

In [6]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output = {
'molid': [0,1],
'prefix': 'Outputs/3_MD_BOMD_Basic_T_cntrl',
'print_every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   1.034080e-01 -4.743546e+02 -4.742512e+02 ||    400.00   1.034080e-01 -1.084874e+02 -1.083840e+02 || 
     2   400.00   1.034080e-01 -4.744150e+02 -4.743116e+02 ||    400.00   1.034080e-01 -1.085724e+02 -1.084690e+02 || 
     3   400.00   1.034080e-01 -4.744873e+02 -4.743839e+02 ||    400.00   1.034080e-01 -1.086660e+02 -1.085626e+02 || 
     4   400.00   1.034080e-01 -4.745664e+02 -4.744630e+02 ||    400.00   1.034080e-01 -1.087615e+02 -1.086581e+02 || 
     5   400.00   1.034080e-01 -4.746485e+02 -4.745451e+02 ||    400.00   1.034080e-01 -1.088549e+02 -1.087515e+02 || 
     6   400.00   1.034080e-01 -4.747311e+02 -4.746277e+02 ||    400.00   1.034080e-01 -1.089439e+02 -1.088405e+02 || 
     7   400.00   1.034080e-01 -4.748123e+02 -4.747089e+02 ||    400.00   1.034080e-01 -1.090271e+02 -1.089237e+02 || 
     8   400.00   1.034080e-01 -4.748909e+02 -4.747875e+02 ||    400.00   1.034080e-01 -1.091037e+02 -1.090003e+

## Langevin Thermostat

In [7]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output = {
'molid': [0,1],
'prefix': 'Outputs/3_MD_BOMD_Langevin',
'print_every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "coordinates": 1,   # write coords; 0 disables
    },
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 50.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   432.20   2.234664e-01 -4.755783e+02 -4.753548e+02 ||    356.97   1.845676e-01 -1.091204e+02 -1.089358e+02 || 
     2   399.08   2.063414e-01 -4.755539e+02 -4.753476e+02 ||    356.73   1.844448e-01 -1.091122e+02 -1.089277e+02 || 
     3   335.79   1.736176e-01 -4.755118e+02 -4.753382e+02 ||    336.85   1.741670e-01 -1.091122e+02 -1.089381e+02 || 
     4   237.24   1.226641e-01 -4.754632e+02 -4.753406e+02 ||    395.47   2.044734e-01 -1.091240e+02 -1.089195e+02 || 
     5   142.16   7.350180e-02 -4.754222e+02 -4.753487e+02 ||    379.64   1.962907e-01 -1.091455e+02 -1.089492e+02 || 
     6    78.83   4.075932e-02 -4.753971e+02 -4.753563e+02 ||    469.38   2.426857e-01 -1.091701e+02 -1.089274e+02 || 
     7    80.99   4.187646e-02 -4.753905e+02 -4.753487e+02 ||    508.55   2.629410e-01 -1.091983e+02 -1.089354e+02 || 
     8   103.66   5.359523e-02 -4.754049e+02 -4.753513e+02 ||    592.90   3.065525e-01 -1.092201e+02 -1.089135e+